## user_reviews
**En este notebook realizaremos la extracción, transformación y carga (ETL) de `user_reviews` Al finalizar el documento podremos exportar los datos para poder iniciar el análisis**

In [1]:
# Importamos librerías
import json             # Módulo de codificador y decodificador JSON
import ast              # Módulo de Árboles de Sintaxis Abstracta
import pandas as pd     # Librería para manipular datasets
import pyarrow as pa    # Útil para operaciones de lectura y escritura de datos
import pyarrow.parquet as pq   # Útil para leer y escribir datos en formato Parquet de manera eficiente
import gzip             # Librería para comprimir y descomprimir datos
import os               # creación de directorios y comprobación de existencia
from textblob import TextBlob

# Extraccion de la informacion

el siguiente código carga los datos del archivo JSON línea por línea, convirtiéndolos en diccionarios de Python y almacenándolos en una lista para su posterior procesamiento.

In [2]:
#Se crea una lista sin elementos con el propósito de almacenar el registro de iteraciones que ocurrirán en el bucle for.
contenido = []

user_review_path = 'C:\\Users\\ahurt\\OneDrive\\Escritorio\\Introduction to IA\\Henry\\GameRecommenderX\\DataSets\\user_reviews.json.gz'
#Creamos bucle que va a recorrer el dataset
for line in gzip.open(user_review_path):
    contenido.append(ast.literal_eval(line.decode('utf-8')))

#Se crea el dataframe en base a la lista
ReviewsDF = pd.DataFrame(contenido)
ReviewsDF

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


# Transformacion, desanidar los datos

El código comienza desanidando los datos de la columna ['reviews'] en un DataFrame utilizando el método explode(), lo que significa que se expanden las listas en esa columna para que cada elemento de la lista tenga su propia fila en el DataFrame resultante. Luego, utiliza pd.json_normalize() para normalizar los datos JSON en cada fila de la columna 'items' expandida, estableciendo el índice como el índice original del DataFrame. Finalmente, concatena horizontalmente los DataFrames resultantes de la normalización con el DataFrame original 'ItemsExplode' mediante pd.concat(), manteniendo así la relación entre los datos normalizados y los datos originales.

In [3]:
# se desanidan los datos de la columna ['items']
ItemsExplode = ReviewsDF.explode(['reviews'])
JsonNormalized = pd.json_normalize(ItemsExplode['reviews']).set_index(ItemsExplode['reviews'].index)
ReviewsDF_unnesting = pd.concat([JsonNormalized, ItemsExplode], axis=1)

In [8]:
# ELimino la columna recien desanidada
ReviewsDF_unnesting = ReviewsDF_unnesting.drop(columns=['reviews'])

In [4]:
ReviewsDF_unnesting

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...,...,...,...,...
25797,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [5]:
# Elimino valores nulos
for column in ReviewsDF_unnesting.columns.to_list():
    ReviewsDF_unnesting = ReviewsDF_unnesting.dropna(subset=[column])


In [6]:
#reinicio el index del dataframe
ReviewsDF_unnesting.reset_index(drop=True, inplace=True)

In [7]:
ReviewsDF_unnesting

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


Mas adelante seguire haciendo transformaciones a este dataframe de momento lo voy a guardar tal como esta

In [8]:
ruta_carpeta = "Data_Extracted"

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# Asigno la ruta donde quiero guardar el parquet con el nombre que va tener
ruta_guardar_parquet = "Data_Extracted/DataFrame_reviews.parquet"

# Transformo a una tabla el DataFrame y luego guardo como archivo Parquet
table = pa.Table.from_pandas(ReviewsDF_unnesting)
pq.write_table(table, ruta_guardar_parquet)